Reference: https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

# Imports

In [ ]:
import sys
import time
from typing import List

import torch
from torch import nn
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split, Subset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import matplotlib.pyplot as plt

# Change system path to base directory.
sys.path.append("..")
from datasets.fake_news_detection import FakeNewsDetectionDataset
from preprocessing.preprocessor import Preprocessor

# Manual set seed for reproducible results
torch.manual_seed(1)

## Read & process dataset

In [ ]:
# Control which linguistic preprocessing steps should run.
preprocessor = Preprocessor(perform_case_folding=True,
                            remove_stop_words=True,
                            remove_punctuation=False,
                            perform_lemmatization=False,
                            perform_stemming=False)

# Generate training and test datasets.
training_dataset = FakeNewsDetectionDataset("../raw_data/fulltrain.csv", transform=preprocessor.process)
test_dataset = FakeNewsDetectionDataset("../raw_data/balancedtest.csv", transform=preprocessor.process)

## Prepare data processing pipelines, generate data batch and iterator

In [ ]:
def yield_tokens(dataset: FakeNewsDetectionDataset) -> List[str]:
    for i in range(len(dataset)):
        sentence, _ = dataset[i]
        yield sentence

vocab = build_vocab_from_iterator(yield_tokens(training_dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
text_pipeline = lambda x: vocab(x)

def collate_batch(batch):
    label_list, text_list = [], []
    for text, label in batch:
        label_list.append(label)
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    # Pads inputs vectors to be of same length (seq_length)
    text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=0)
    # Label list size: [batch_size]
    # Text list size: [batch_size, seq_length]
    return label_list.to(device), text_list.to(device)

## Training

### Model class

In [ ]:
hidden_size = 128

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, batch_size):
        super(TextClassificationModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(input_size=embed_dim,
                            hidden_size=hidden_size,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)

        self.fc = nn.Linear(2*hidden_size, 4)
        self.init_weights()
        self.reset_h_c(batch_size)

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def reset_h_c(self, batch_size):
        # hidden state and cell state size:
        # [num_layers, batch_size, hidden_size]
        # not sure but i think num_layers = 2 if bidirectional else 1
        self.h = torch.zeros(2, batch_size, hidden_size)
        self.c = torch.zeros(2, batch_size, hidden_size)
        self.h = self.h.to(device)
        self.c = self.c.to(device)

    def forward(self, text):   
        self.h = self.h.detach()
        self.c = self.c.detach()
        self.h = self.h.requires_grad_()
        self.c = self.c.requires_grad_()
        # Embedding vector size: [batch_size, seq_length, embed_dim]
        g_seq = self.embedding(text)
        # LSTM output size: [batch_size, seq_length, num_layers*hidden_size]
        h_seq, (self.h, self.c)  =  self.lstm( g_seq , (self.h, self.c) )
        # LSTM final state vector size: [batch_size, num_layers*hidden_size]
        final_state = h_seq[:,-1,:]
        # Linear layer output size: [batch_size, 4]
        score_seq = self.fc(h_seq[:,-1,:])
        return score_seq


### Define train/eval functions, split training data

In [ ]:
# Hyperparameters
EPOCHS = 1 # epoch
LR = 0.005  # learning rate
BATCH_SIZE = 64 # batch size for training

vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, BATCH_SIZE).to(device)

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    model.reset_h_c(BATCH_SIZE)

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
num_train = int(len(training_dataset) * 0.95)
split_train_: Subset[FakeNewsDetectionDataset]
split_valid_: Subset[FakeNewsDetectionDataset]
split_train_, split_valid_ = random_split(training_dataset, [num_train, len(training_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

### Training Loop

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

In [ ]:
# Save trained model
torch.save(model.state_dict(), 'LSTMModel.pt')

In [ ]:
# Load saved model to skip training
model.load_state_dict(torch.load('LSTMModel.pt'))

### Evaluating on test set

In [ ]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

### Checking relationships between labels for documents

In [ ]:
CATEGORIES = { 1: 'Satire', 2: 'Hoax', 3: 'Propaganda', 4: 'Reliable news' }

reliable, hoax, satire, propaganda = [], [], [], []

all_documents_dataloader = DataLoader(training_dataset, batch_size=1,
                             shuffle=True, collate_fn=collate_batch)

with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(all_documents_dataloader):
        output = model(text, offsets)
        arr = output.numpy()[0]
        reliable.append(arr[3])
        hoax.append(arr[1])
        satire.append(arr[0])
        propaganda.append(arr[2])

In [ ]:
plt.scatter(reliable, hoax)
plt.xlabel('reliable')
plt.ylabel('hoax')
plt.title('Reliable vs hoax - documents')
plt.show()

plt.scatter(reliable, satire)
plt.xlabel('reliable')
plt.ylabel('satire')
plt.title('Reliable vs satire - documents')
plt.show()

plt.scatter(reliable, propaganda)
plt.title('Reliable vs propaganda - documents')
plt.xlabel('reliable')
plt.ylabel('propaganda')
plt.show()

plt.scatter(hoax, propaganda)
plt.title('Hoax vs propaganda - documents')
plt.xlabel('hoax')
plt.ylabel('propaganda')
plt.show()

plt.scatter(hoax, satire)
plt.title('Hoax vs satire - documents')
plt.xlabel('hoax')
plt.ylabel('satire')
plt.show()

plt.scatter(satire, propaganda)
plt.title('Satire vs propaganda - documents')
plt.xlabel('satire')
plt.ylabel('propaganda')
plt.show()

### Checking relationships between labels for words

In [ ]:
words = vocab.get_itos()

def collate_word(word):
    text_list, offsets = [], [0]
    processed_text = torch.tensor(text_pipeline(word), dtype=torch.int64)
    text_list.append(processed_text)
    offsets.append(processed_text.size(0))
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list.to(device), offsets.to(device)

reliable, hoax, satire, propaganda = [], [], [], []

with torch.no_grad():
    for word in words:
        text, offsets = collate_word(list(word))
        output = model(text, offsets)
        arr = output.numpy()[0]
        reliable.append(arr[3])
        hoax.append(arr[1])
        satire.append(arr[0])
        propaganda.append(arr[2])

In [ ]:
plt.scatter(reliable, hoax)
plt.xlabel('reliable')
plt.ylabel('hoax')
plt.title('Reliable vs hoax - words')
plt.show()

plt.scatter(reliable, satire)
plt.xlabel('reliable')
plt.ylabel('satire')
plt.title('Reliable vs satire - words')
plt.show()

plt.scatter(reliable, propaganda)
plt.title('Reliable vs propaganda - words')
plt.xlabel('reliable')
plt.ylabel('propaganda')
plt.show()

plt.scatter(hoax, propaganda)
plt.title('Hoax vs propaganda - words')
plt.xlabel('hoax')
plt.ylabel('propaganda')
plt.show()

plt.scatter(hoax, satire)
plt.title('Hoax vs satire - words')
plt.xlabel('hoax')
plt.ylabel('satire')
plt.show()

plt.scatter(satire, propaganda)
plt.title('Satire vs propaganda - words')
plt.xlabel('satire')
plt.ylabel('propaganda')
plt.show()